# Pre-processing_final_1
This notebook merges 3 preprocessed datasets and prepares for EDA_daily and geospatial analysis

In [2]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sbs
import geopandas as gpd
import folium 

In [3]:
from pyspark.sql import SparkSession

# Create a spark session with increased memory allocation
spark = (
    SparkSession.builder.appName("ADS Project1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "8g")  # Set the driver memory to 8GB
    .config("spark.executor.memory", "8g")  # Set the executor memory to 8GB
    .getOrCreate()
)

24/08/30 11:41:35 WARN Utils: Your hostname, Hanshis-Laptop.local resolves to a loopback address: 127.0.0.1; using 172.16.119.21 instead (on interface en0)
24/08/30 11:41:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/30 11:41:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/30 11:41:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Datasets

In [4]:
# read the data from preprocessed tlc data
tdf = spark.read.parquet("../data/raw/tlc_df.parquet") 

In [5]:
# read the preprocessed weather data
wdf = spark.read.csv("../data/raw/NYC_weather_raw.csv", header=True, inferSchema=True)

In [6]:
# read the preprocessed event data
edf = spark.read.parquet("../data/raw/NYC_Permitted_Event_Information_Historical.parquet") 

## Aggregation

In [7]:
# Load the zones DataFrame
zones = spark.read.csv("../data/landing/external/taxi_zones.csv", header=True, inferSchema=True)

In [8]:
# Join the tdf DataFrame with zones to get the borough for PULocationID
tdf = tdf.join(zones.select('LocationID', 'borough').withColumnRenamed('borough', 'PUBorough'),
               tdf['PULocationID'] == zones['LocationID'], 'left').drop('LocationID')

# Join the tdf DataFrame with zones to get the borough for DOLocationID
tdf = tdf.join(zones.select('LocationID', 'borough').withColumnRenamed('borough', 'DOBorough'),
               tdf['DOLocationID'] == zones['LocationID'], 'left').drop('LocationID')


In [11]:
# Convert pickup_date and pickup_hour to a timestamp and create Time column
tdf = tdf.withColumn(
    'Time', 
    to_timestamp(concat_ws(' ', col('pickup_date')))
)

df_daily_agg = tdf.groupBy(['pickup_date', "PUBorough"]).agg({
    '*': 'count',
})

In [12]:
# Aggregate the number of events per date and borough
edf_daily = edf.groupBy('Start Date', 'Event Borough').agg(count('Event ID').alias('Number of Events'))

## Merge

In [13]:
# Step 1: Create unified datetime columns in each dataset

# For tdf (taxi dataset)
df_daily_agg = df_daily_agg.withColumn("datetime", to_timestamp(concat_ws(" ", col("pickup_date")), "yyyy-MM-dd"))

# For edf (event dataset)
edf_daily = edf_daily.withColumn("datetime", to_timestamp(concat_ws(" ", col("Start Date")), "yyyy-MM-dd"))
edf_daily = edf_daily.withColumnRenamed("Event Borough", "PUBorough")

# For wdf (weather dataset)
wdf = wdf.withColumn("datetime", to_timestamp(col("DATE"), "yyyy-MM-dd"))

# Step 2: Join tdf (taxi dataset) with edf (event dataset) on datetime and borough
tdf_edf = df_daily_agg.join(edf_daily, on=["datetime", "PUBorough"], how="left")

# Step 3: Join the result with the weather dataset on datetime only (since weather data is from one station)
final_df = tdf_edf.join(wdf, 
                        on="datetime", 
                        how="left")


In [ ]:
# Group by 'pickup_date' and 'PUBorough' to get the sum of trips and average weather data
daily_demand = clean_df.groupby(['pickup_date', 'PUBorough']).agg({
    'daily_trip_count': 'sum',
    'TMP': 'mean',
    'VIS': 'mean',
    'Number of Events': 'sum'
}).reset_index()

# Rename the columns to match the desired output
daily_demand.rename(columns={
    'daily_trip_count': 'total_daily_trips',
    'TMP': 'avg_temperature',
    'VIS': 'avg_visibility',
    'Number of Events': 'total_events'
}, inplace=True)

# Round the average temperature and visibility to 1 decimal place
daily_demand['avg_temperature'] = daily_demand['avg_temperature'].round(1)
daily_demand['avg_visibility'] = daily_demand['avg_visibility'].round(1)

# Show the first 5 rows of the aggregated result
print(daily_demand.head(5))
